In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
%cd /content/drive/MyDrive/Colab\ Notebooks/skema/data/ASKEM\ Grounding\ Annotations/
!ls

/content/drive/MyDrive/Colab Notebooks/skema/data/ASKEM Grounding Annotations
annotation_instructions.md
grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx
mira_dkg_epi_pretty.json


In [36]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/ASKEM Grounding Annotations"
filename = "grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx"
json_filename = "mira_dkg_epi_pretty.json"
df = pd.read_excel(os.path.join(path, filename))

In [48]:
import json
with open(os.path.join(path, json_filename), "r") as f:
  ontology = json.load( f)
ontology[:4]

[{'id': 'apollosv:00000000',
  'name': 'purely intentional entity',
  'description': 'A generically dependent continuant which derives its existence and its entire endowment from an intending experience of consciousness (an act) that is laden with determinate, uniformily structured content.',
  'synonyms': []},
 {'id': 'apollosv:00000001',
  'name': 'epidemic model',
  'description': 'A disease transmission model whose concretization is realized as a simulation of infectious disease epidemic or infectious disease pandemic.',
  'synonyms': []},
 {'id': 'apollosv:00000002',
  'name': 'basic reproduction number',
  'description': 'The average number of transmissions of an infectious agent from an infectious organism to a susceptible organism within a completely susceptible population.',
  'synonyms': []},
 {'id': 'apollosv:00000003',
  'name': 'population immunity survey',
  'description': 'A population survey that assesses the level of immunity against a particular pathogen in a specifie

In [49]:
len(ontology)

80738

In [37]:
df['Text'] = df['Text'].fillna("NA")
df['Role'] = df['Role'].fillna("NA")
df['Context'] = df['Context'].fillna("NA")
df['Annotation'] = df['Annotation'].fillna("")
df['Identified_type'] = df['Identified_type'].fillna("")
df['Comments'] = df['Comments'].fillna("")

In [38]:
df.head()

,Text,Role,Context,Candidate ID,Candidate Text,Score,Annotation,Comments,Identified_type
0,for running simulations,context,for running simulations,apollosv:00000015,simulation run length,0.786254,G,,
1,NA,NA,NA,apollosv:00000066,simulation of time step,0.757462,G,,
2,NA,NA,NA,apollosv:00000042,simulation of disease,0.716728,G,,
3,NA,NA,NA,apollosv:00000007,simulation software application,0.715411,G,,
4,NA,NA,NA,apollosv:00000070,simulation,0.704855,X,,


In [39]:
prev_text = ""
prev_role = ""
prev_context = ""
for index, this_row in df.iterrows():
  this_row_list = this_row.values.tolist()
  if this_row['Text'] != "NA" and this_row['Role'] != "NA" and this_row['Context'] != "NA":
    prev_text = this_row['Text']
    prev_role = this_row['Role']
    prev_context = this_row['Context']
  else:
    df.at[index, 'Text'] = prev_text
    df.at[index, 'Role'] = prev_role
    df.at[index, 'Context'] = prev_context

In [42]:
df.head()

,Text,Role,Context,Candidate ID,Candidate Text,Score,Annotation,Comments,Identified_type
0,for running simulations,context,for running simulations,apollosv:00000015,simulation run length,0.786254,G,,
1,for running simulations,context,for running simulations,apollosv:00000066,simulation of time step,0.757462,G,,
2,for running simulations,context,for running simulations,apollosv:00000042,simulation of disease,0.716728,G,,
3,for running simulations,context,for running simulations,apollosv:00000007,simulation software application,0.715411,G,,
4,for running simulations,context,for running simulations,apollosv:00000070,simulation,0.704855,X,,


In [43]:
df["kg_match"] = ""

In [46]:
len(df['Text'].unique())

294

In [90]:
def get_matching_entry(ontology, text, context, candidate_ids):
  matching_candidates = {}
  for entry in ontology:
    #print(entry)
    desc = ""
    if "description" in entry.keys():
      desc = entry['description']
    id = entry['id']
    name = entry['name']
    e = id +","+name+","+desc
    if text in entry['name'] and entry['id'] not in candidate_ids:
      #print(text, entry)
      matching_candidates[entry['id'] ] = e
    if context in entry['name'] and entry['id'] not in candidate_ids:
      #print(context, entry)
      matching_candidates[entry['id'] ] = e
    if "description" in entry.keys():
      if text in entry['description'] and entry['id'] not in candidate_ids:
        #print(text, entry)
        matching_candidates[entry['id'] ] = e
      if context in entry['description'] and entry['id'] not in candidate_ids:
        #print(context, entry)
        matching_candidates[entry['id'] ] = e
  return matching_candidates

In [91]:
for index, this_row in df.iterrows():
  matching_candidates = []
  candidate_ids = df['Candidate ID'][df['Text'] == this_row['Text']].to_list()
  # print(candidate_ids)
  matching_candidates = get_matching_entry(ontology, this_row['Text'], this_row['Context'], candidate_ids)
  if len(matching_candidates) > 0:
    print(len(matching_candidates))
    df.at[index, "kg_match"] = ";".join(matching_candidates.values())

1307
1307
1307
1307
1307
1307
1307
1307
1307
1307
1307
1307
1307
1307
1307
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
29290
29290
29290
29290
29290
29290
29290
29290
29290
29290
29290
29290
29290
29290
29290
61
61
61
61
61
61
61
61
61
61
61
61
61
61
61
344
344
344
344
344
344
344
344
344
344
344
344
344
344
344
48
48
48
48
48
48
48
48
48
917
917
917
917
917
917
917
917
917
917
917
917
917
917
917
353
231
231
231
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
158
158
158
158
158
158
158
158
158
158
158
158
158
158
158
13
13
13
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
136
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
22
22
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2


In [92]:
df.to_excel(os.path.join(path, "grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data-ontology_matches.xlsx"))